# CNN vs SVM CAPTCHA Classifier

In [2]:
! pip install claptcha

### Import necessary libraries

In [47]:
import cv2                    # OpenCV for computer vision tasks
import glob                   # File system pattern matching
import joblib                 # Joblib for efficient Python object serialization
import os                     # Operating system-related functions
import numpy as np            # NumPy for numerical operations
import pandas as pd           # Pandas for data manipulation and analysis
import random                 # Random number generation
import requests               # HTTP requests library
import shutil                 # File operations (copy, move, etc.)
import string                 # String manipulation functions

from claptcha import Claptcha           # Claptcha for generating CAPTCHA images
from keras.models import Sequential     # Keras deep learning framework for neural networks
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization  # Keras layers for building neural networks
from keras.preprocessing.image import ImageDataGenerator  # Keras image data augmentation
from PIL import Image                   # Python Imaging Library for image processing
from skimage.feature import hog         # Histogram of Oriented Gradients feature extraction
from sklearn.model_selection import train_test_split, GridSearchCV  # scikit-learn for machine learning tools
from sklearn.metrics import accuracy_score, classification_report  # Model evaluation metrics
from sklearn.svm import SVC             # Support Vector Machine classifier

### Define Constants

In [67]:
train_dir = "train_dir"
val_dir = "val_dir"
font = "https://github.com/opensourcedesign/fonts/raw/master/gnu-freefont_freemono/FreeMono.ttf"
font_path = font.split("/")[-1]           # Extract the font file name from the URL
captcha_length = 4                        # Length of the CAPTCHA text
char_width = 37.5
image_size = (int(captcha_length * char_width), 90)
image_margin = (0, 0)
total_images = 6000                       # Total number of CAPTCHA images to generate
test_set_size = int(0.15 * total_images)  # Size of the test set
val_set_size = int(0.2 * (total_images - test_set_size))  # Size of the val set
svm_model = 'svm_model.pkl'               # File name to save the trained SVM model
cnn_model = 'cnn_model.hdf5'

class_mapping = [str(i) for i in range(10)]
class_mapping.extend([chr(ord('A') + i) for i in range(26)])

# CAPTCHA Generation and Preprocessing
In this section, we will generate CAPTCHAs and preprocess them for training a machine learning model. We'll follow these steps:


### 1. **Font Download**: Download a font for creating CAPTCHAs.
We need a specific font to create our CAPTCHAs. We'll download the font from a public source.

In [5]:
# Helper function to download the font
def download_font(font_url, font_path):
    if not os.path.isfile(font_path):
        response = requests.get(font_url)
        if response.status_code == 200:
            with open(font_path, 'wb') as font_file:
                font_file.write(response.content)
        else:
            raise Exception("Failed to download the font file.")

download_font(font, font_path)

### 2. **CAPTCHA Generation**: Generate random CAPTCHA text and create CAPTCHA images.

In [6]:
def random_string(length):
    """Helper function to generate random strings"""
    rndLetters = (random.sample(string.ascii_uppercase + string.digits, length))
    return "".join(rndLetters)

In [7]:
def process_captcha_image(image):
    """Function to preprocess individual letters of CAPTCHA images"""
    gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.threshold(gray_scale, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    return image

In [8]:
def saveimage(letter_text,letter_image, value, folder):
    '''A function to save the images'''
    os.makedirs(folder, exist_ok=True)  # Create folder
    path = os.path.join(folder, letter_text)
    if not os.path.exists(path):
        os.makedirs(path)
    p = os.path.join(path, f"{value}.png")
    cv2.imwrite(p, letter_image)

In [54]:
data = []
labels = []
test_data = {}
available_width = image_size[0] - image_margin[0] / 2
character_width = available_width / captcha_length

# Generate CAPTCHA images for training and test sets
for i in range(total_images):
    text = random_string(captcha_length)
    c = Claptcha(text, font_path, image_size, resample=Image.BICUBIC, noise=0.2)
    c.margin = image_margin
    _, image = c.image

    # Apply preprocessing steps to the image
    processed_image_array = process_captcha_image(np.array(image))

    # Save the preprocessed image
    if i < test_set_size:
        # Save the preprocessed image to the test data dictionary
        test_data[text + str(i)] = processed_image_array

    #Slicing the image and saving the image file
    else:
        for j in range(captcha_length):
            start_col = int(image_margin[0] / 2 + character_width * j)
            end_col = int(start_col + character_width)
            sliced_image = processed_image_array[0:image_size[1],start_col:end_col]
            feature_vector = hog(sliced_image, pixels_per_cell=(8, 8))
            data.append(feature_vector)
            label = class_mapping.index(text[j])
            labels.append(label)
            if i < test_set_size + val_set_size:
                saveimage(text[j], sliced_image, f'{text}[{j}]', val_dir)
            else:
                saveimage(text[j], sliced_image, f'{text}[{j}]', train_dir)

## 4. Split the data into training and testing sets
Split the dataset into training and testing sets, train a Support Vector Machine (SVM) classifier, and evaluate its performance.

In [55]:
X_train, X_test, y_train, y_test = train_test_split(np.array(data), labels, test_size=0.2, random_state=42)

## 4. Hyperparameter Tuning
To find the optimal combination of hyperparameters for your SVM model, we can use techniques like grid search or random search.

In [ ]:
# @title
# Define a parameter grid to search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1]
}

# Create the grid search model
grid_search = GridSearchCV(SVC(), param_grid, cv=5)

# Fit the grid search to your training data
grid_search.fit(X_train[:2000], y_train[:2000])

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}


## 2. Initialize the SVM classifier and train it

In [56]:
# Create an SVM classifier
svm_clf = SVC(kernel='poly', gamma=0.1, C=10)

# Train the SVM on your training data
svm_clf.fit(X_train, y_train)

# Save the trained SVM model to a file
joblib.dump(svm_clf, svm_model)

y_pred = svm_clf.predict(X_test)

# Calculate accuracy and generate a classification report

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.9637254901960784


In [57]:
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=class_mapping))

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.94      0.95       118
           1       0.93      0.93      0.93       114
           2       0.97      0.97      0.97       110
           3       0.99      0.95      0.97       143
           4       0.93      0.96      0.95       119
           5       0.98      0.97      0.97        97
           6       0.96      0.98      0.97       108
           7       0.99      0.97      0.98       114
           8       0.91      0.94      0.93       114
           9       0.97      0.99      0.98       105
           A       0.98      0.99      0.98       129
           B       0.91      0.98      0.94       117
           C       1.00      0.96      0.98       133
           D       0.94      0.95      0.95       106
           E       0.94      0.95      0.95       120
           F       0.97      0.95      0.96       117
           G       0.97      0.99      0.98       114
   

In [58]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally by up to 10%
    height_shift_range=0.1,  # Randomly shift images vertically by up to 10%
    shear_range=0.2,  # Shear intensity
    zoom_range=0.2,  # Randomly zoom in by up to 20%
    horizontal_flip=False,  # Don't flip horizontally
    fill_mode='nearest'  # Fill missing pixels with the nearest value
)

# Training data generation - from directory train_dir
training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size = (32, 32),
    class_mode = 'categorical',color_mode='grayscale'
    )

# Data Augmentation
val_datagen = ImageDataGenerator()

# Validation data generation - from directory val_dir
validation_set = val_datagen.flow_from_directory(
    val_dir,
    target_size = (32, 32),
    class_mode = 'categorical',color_mode='grayscale'
    )

Found 16280 images belonging to 36 classes.
Found 4080 images belonging to 36 classes.


In [59]:
cnn_clf = Sequential([
    Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 1)),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(32, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(36, activation='softmax')
])

# Compile the model with adjusted learning rate
cnn_clf.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

## Fitting the model
history = cnn_clf.fit(training_set,
                      validation_data=validation_set,
                      batch_size=64,
                      epochs=10)

Epoch 1/10
509/509 [==============================] - 47s 87ms/step - loss: 2.2413 - accuracy: 0.3863 - val_loss: 1.1007 - val_accuracy: 0.6699
Epoch 2/10
509/509 [==============================] - 45s 89ms/step - loss: 1.1561 - accuracy: 0.6724 - val_loss: 0.5492 - val_accuracy: 0.8404
Epoch 3/10
509/509 [==============================] - 46s 91ms/step - loss: 0.7844 - accuracy: 0.7706 - val_loss: 0.4776 - val_accuracy: 0.8534
Epoch 4/10
509/509 [==============================] - 45s 88ms/step - loss: 0.5908 - accuracy: 0.8213 - val_loss: 0.3210 - val_accuracy: 0.9015
Epoch 5/10
509/509 [==============================] - 43s 85ms/step - loss: 0.4992 - accuracy: 0.8455 - val_loss: 0.3136 - val_accuracy: 0.9012
Epoch 6/10
509/509 [==============================] - 44s 87ms/step - loss: 0.4396 - accuracy: 0.8639 - val_loss: 0.1652 - val_accuracy: 0.9466
Epoch 7/10
509/509 [==============================] - 42s 83ms/step - loss: 0.4119 - accuracy: 0.8734 - val_loss: 0.1805 - val_accuracy:

In [63]:
## Hierarchical Data format file to save the model. So that we dont have to run the entire sequence of program again.
cnn_clf.save(cnn_model)

In [61]:
#Get list of all Test files from the mapping of the classes
d = training_set.class_indices
dict_class = {v:k for k,v in d.items()}

In [64]:
# Counter to count the number of correct predictions
cnn_count = 0
svm_count = 0

## Predict function to test all the images from the Test_data directory
for value, image in test_data.items():
    value = value[:4]
    cnn_combine = []
    svm_combine = []
    #Slicing the image
    for i in range(captcha_length):
        start_col = int(image_margin[0]/2 + character_width * i)
        end_col = int(start_col + character_width)
        sliced_image = image[0:image_size[0], start_col:end_col]
        feature_vector = hog(sliced_image, pixels_per_cell=(8, 8))
        resized = cv2.resize(sliced_image,(32,32))
        img= np.expand_dims(resized,axis=2)
        img = np.expand_dims(img,axis=0)
        class_probabilities = cnn_clf.predict(img)
        svm_combine.append(dict_class[svm_clf.predict([feature_vector])[0]])
        cnn_combine.append(dict_class[np.argmax(class_probabilities, axis=1)[0]])

    cnn_predict = ''.join(cnn_combine)
    svm_predict = ''.join(svm_combine)
    print(f'CNN prediction: {cnn_predict}')
    print(f'SVM prediction: {svm_predict}')
    print(f'Actual value  : {value}')
    print("------------")
    # Increment the counter if the predicted and actual captcha letters match
    if value == cnn_predict:
        cnn_count += 1
    if value == svm_predict:
        svm_count += 1

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 26ms/step
CNN prediction: 91UF
SVM prediction: 91UF
Actual value  : 91UF
------------
1/1 [==============================] - 0s 28ms/step
CNN prediction: HSP5
SVM prediction: H0P5
Actual value  : H0P5
------------
1/1 [==============================] - 0s 30ms/step
CNN prediction: GJAV
SVM prediction: GJAV
Actual value  : GJAV
------------
1/1 [==============================] - 0s 25ms/step
CNN prediction: 5F2O
SVM prediction: 5F2O
Actual value  : 5F2O
------------
1/1 [==============================] - 0s 23ms/step
CNN prediction: GPHL
SVM prediction: GPHL
Actual value  : GPHL
------------
1/1 [==============================] - 0s 25ms/step
CNN prediction: FNT3
SVM prediction: FNT3
Actual value  : FNT3
------------
1/1 [==============================] - 0s 29ms/step
CNN prediction: IXWH
SVM prediction: IXWH
Actual value  : IXWH
------------
1/1 [==============================] - 0s 31ms/step
C

In [65]:
# Number of correct predictions is stored in count variable
cnn_accuracy = cnn_count/test_set_size
svm_accuracy = svm_count/test_set_size
print(f'CNN accuracy: {cnn_accuracy}')
print(f'SVM accuracy: {svm_accuracy}')

CNN accuracy: 0.9055555555555556
SVM accuracy: 0.8655555555555555
